In [ ]:
import panel as pn
pn.extension()

The ``VideoStream`` widget displays a video from a local stream (for example from a webcam).

For more information about listening to widget events and laying out widgets refer to the [widgets user guide](../../user_guide/Widgets.ipynb). Alternatively you can learn how to build GUIs by declaring parameters independently of any specific widgets in the [param user guide](../../user_guide/Param.ipynb). To express interactivity entirely using Javascript without the need for a Python server take a look at the [links user guide](../../user_guide/Param.ipynb).

#### Parameters:

For layout and styling related parameters see the [customization user guide](../../user_guide/Customization.ipynb).

* **``paused``** (boolean): Whether the video stream is paused
* **``snapshot``** (boolean): On change a snapshot of the current frame is generated
* **``value``** (string): String representation of the current snapshot

You need at least a valid stream input to test the widget
___

In [ ]:
stream_vid = pn.widgets.VideoStream(height=480)
stream_vid

In [ ]:
stream_vid.snapshot = not stream_vid.snapshot

In [ ]:
pn.pane.HTML('<img src="' + stream_vid.value + '" />', height=480)

We can synchronize the widget with a custom parameterized class to take screenshots at regular time steps

In [ ]:
import param
class VideoSnap(param.Parameterized):
    screenshot = param.String()
    paused = param.Boolean(default=False)
    
    def __init__(self, **params):
        super(VideoSnap,self).__init__(**params)
        self._video = pn.widgets.VideoStream(width=640, height=480)
        self._video.paused = self.paused
        self._video.link(self, value='screenshot')
        
    @param.depends('screenshot', watch=True)
    def display_screenshot(self):
        return pn.pane.HTML('<img src="' + self.screenshot + '" />')
    
    def panel(self):
        return pn.Row(self._video, self.display_screenshot)
    
    def take_screenshot(self):
        self._video.snapshot = not self._video.snapshot
    
    @param.depends('paused', watch=True)
    def on_pause(self):
        self._video.paused = self.paused

vid = VideoSnap()
pan_vid = vid.panel()
pan_vid.add_periodic_callback(vid.take_screenshot, 100)
pan_vid

In [ ]:
vid.paused = True